In [58]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression

In [59]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

Загрузите данные и посчитайте модели линейной регрессии для 50 зданий по ансамблю регрессионных моделей: в первой модели весь оптимальный набор метеорологических данных, во второй - дни недели и праздники, в третьей - недели года, в четвертой - месяцы. Финальное значение показателя рассчитайте как взвешенное арифметическое показателей всех моделей, взяв веса для первой и второй модели как 3/8, а для третьей и четвертой - как 1/8.

Загрузите данные решения, посчитайте значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели, и выгрузите результат в виде CSV-файла (submission.csv).

Данные:

http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz

http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz

http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz

http://video.ittensive.com/machine-learning/ashrae/test.csv.gz

http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz
### Сколько строк в итоговом файле имеют ненулевое значение для показателя meter_reading

In [60]:
buildings = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz',
                        usecols=["site_id", "building_id"])
weather = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz')

# http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
energy = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz')
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
results = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/test.csv.gz')
results = results[results['building_id'] < 50]
del buildings



In [61]:
energy = energy[energy['building_id'] < 50]
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)

In [62]:
energy = reduce_mem_usage(energy)
energy = energy.drop('meter',axis=1)
print (energy.info())

Потребление памяти меньше на 28.46 Мб (минус 71.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432456 entries, 0 to 432455
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           432456 non-null  datetime64[ns]
 1   site_id             432456 non-null  int8          
 2   building_id         432456 non-null  int8          
 3   meter_reading       432456 non-null  float16       
 4   air_temperature     432312 non-null  float16       
 5   cloud_coverage      243994 non-null  float16       
 6   dew_temperature     432312 non-null  float16       
 7   precip_depth_1_hr   432408 non-null  float16       
 8   sea_level_pressure  428287 non-null  float16       
 9   wind_direction      420144 non-null  float16       
 10  wind_speed          432456 non-null  float16       
dtypes: datetime64[ns](1), float16(8), int8(2)
memory usage: 10.7 MB
None


In [63]:
interpolate_columns = ['air_temperature','dew_temperature','wind_speed',
                       'cloud_coverage','sea_level_pressure']
energy = energy.drop(['precip_depth_1_hr','wind_direction'],axis=1)
for col in interpolate_columns:
  energy[col] = energy[col].interpolate(limit_direction='both',kind='cubic')


In [64]:
energy['air_temperature_diff1'] = energy['air_temperature'].diff()
energy.at[0,'air_temperature_diff1'] = energy.at[1,'air_temperature_diff1']
energy['air_temperature_diff2'] = energy['air_temperature_diff1'].diff()
energy.at[0,'air_temperature_diff2'] = energy.at[1,'air_temperature_diff2']
energy = energy.drop('site_id',axis=1)

In [65]:
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
energy['week'] = energy['timestamp'].dt.week.astype("int8")
energy['month'] = energy['timestamp'].dt.month.astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype("int8")
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
for week in range(1,54):
    energy['is_w' + str(week)] = energy['week'].isin([week]).astype("int8")
for month in range(1,13):
    energy['is_m' + str(month)] = energy['month'].isin([month]).astype("int8")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [66]:
energy.columns

Index(['timestamp', 'building_id', 'meter_reading', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'sea_level_pressure', 'wind_speed',
       'air_temperature_diff1', 'air_temperature_diff2', 'hour', 'weekday',
       'week', 'month', 'date', 'is_holiday', 'meter_reading_log', 'is_wday0',
       'is_wday1', 'is_wday2', 'is_wday3', 'is_wday4', 'is_wday5', 'is_wday6',
       'is_w1', 'is_w2', 'is_w3', 'is_w4', 'is_w5', 'is_w6', 'is_w7', 'is_w8',
       'is_w9', 'is_w10', 'is_w11', 'is_w12', 'is_w13', 'is_w14', 'is_w15',
       'is_w16', 'is_w17', 'is_w18', 'is_w19', 'is_w20', 'is_w21', 'is_w22',
       'is_w23', 'is_w24', 'is_w25', 'is_w26', 'is_w27', 'is_w28', 'is_w29',
       'is_w30', 'is_w31', 'is_w32', 'is_w33', 'is_w34', 'is_w35', 'is_w36',
       'is_w37', 'is_w38', 'is_w39', 'is_w40', 'is_w41', 'is_w42', 'is_w43',
       'is_w44', 'is_w45', 'is_w46', 'is_w47', 'is_w48', 'is_w49', 'is_w50',
       'is_w51', 'is_w52', 'is_w53', 'is_m1', 'is_m2', 'is_m3', 'is_m4',
  

In [67]:
lr_columns = ['building_id', 'meter_reading_log', 'hour', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'sea_level_pressure', 'wind_speed',
       'air_temperature_diff1', 'air_temperature_diff2', 'weekday',
       'week', 'is_holiday', 'is_wday0',
       'is_wday1', 'is_wday2', 'is_wday3', 'is_wday4', 'is_wday5', 'is_wday6',
       'is_w1', 'is_w2', 'is_w3', 'is_w4', 'is_w5', 'is_w6', 'is_w7', 'is_w8',
       'is_w9', 'is_w10', 'is_w11', 'is_w12', 'is_w13', 'is_w14', 'is_w15',
       'is_w16', 'is_w17', 'is_w18', 'is_w19', 'is_w20', 'is_w21', 'is_w22',
       'is_w23', 'is_w24', 'is_w25', 'is_w26', 'is_w27', 'is_w28', 'is_w29',
       'is_w30', 'is_w31', 'is_w32', 'is_w33', 'is_w34', 'is_w35', 'is_w36',
       'is_w37', 'is_w38', 'is_w39', 'is_w40', 'is_w41', 'is_w42', 'is_w43',
       'is_w44', 'is_w45', 'is_w46', 'is_w47', 'is_w48', 'is_w49', 'is_w50',
       'is_w51', 'is_w52', 'is_w53', 'month', 'is_m1', 'is_m2', 'is_m3',
       'is_m4', 'is_m5', 'is_m6', 'is_m7', 'is_m8', 'is_m9', 'is_m10',
       'is_m11', 'is_m12']


In [68]:
from sklearn.model_selection import train_test_split
energy_train,energy_test = train_test_split(energy[energy['meter_reading'] > 0],test_size=0.2)

In [69]:
energy_train_lr = pd.DataFrame(energy_train,columns=lr_columns)

In [70]:
hours = range(0,24)
buildings = range(0,energy_train_lr['building_id'].max() + 1)
energy_lr = [[]] * len(buildings)
for building in buildings:
  energy_lr[building] = [[]] * len(hours)
  energy_train_b = energy_train_lr[energy_train_lr['building_id'] == building]
  for hour in hours:
    energy_train_bh = energy_train_b[energy_train_b['hour'] == hour]
    y = energy_train_bh['meter_reading_log']
    x = energy_train_bh.drop(['meter_reading_log','hour','building_id'],axis=1)
    model = LinearRegression(fit_intercept=False).fit(x,y)
    energy_lr[building][hour] = model.coef_
    energy_lr[building][hour] = np.append(energy_lr[building][hour],model.intercept_)
print(energy_lr[0])

[array([-3.43998522e-03, -1.57582238e-02,  2.11236943e-02, -1.23891309e-02,
        2.42795795e-03, -4.12735343e-03, -4.11000103e-03,  6.73379183e-01,
        5.46907671e-02, -1.32365853e-01,  3.00615501e+00,  2.30940366e+00,
        1.62960279e+00,  9.30225551e-01,  2.57481843e-01, -4.24155712e-01,
       -1.09923100e+00, -6.13927841e-06, -1.31130219e-06,  6.55651093e-06,
        1.01327896e-05,  1.53779984e-05,  9.11951065e-06, -1.00135803e-05,
       -9.53674316e-06, -9.05990601e-06, -2.86102295e-06,  1.90734863e-06,
       -1.19209290e-06, -1.19209290e-06,  6.19888306e-06,  2.38418579e-07,
       -2.86102295e-06,  4.29153442e-06, -9.59634781e-06,  2.02655792e-06,
        1.55968916e+00,  1.32637572e+00,  1.10973656e+00,  1.17383933e+00,
        1.21590662e+00,  1.07284319e+00,  9.33798790e-01,  9.53229129e-01,
        1.06143141e+00,  9.26291227e-01,  8.28501344e-01,  5.95813096e-01,
        5.41476369e-01,  5.74400902e-01,  2.91855454e-01,  1.47140205e-01,
        1.44664526e-01, 

In [71]:
def calculate_model(x):
  lr = -1
  model = energy_lr[x.building_id][x.hour] 
  if len(model) > 0:
    lr = np.sum([x[c] * model[i] for i,c in enumerate(lr_columns[3:])])
    lr += model[len(lr_columns) - 3]
    lr = np.exp(lr)
  if lr < 0 or lr != lr or lr*lr == lr:
    lr = 0
  x['meter_reading'] = lr
  return x

In [72]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1051200 entries, 0 to 2260073
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   row_id       1051200 non-null  int64 
 1   building_id  1051200 non-null  int64 
 2   meter        1051200 non-null  int64 
 3   timestamp    1051200 non-null  object
dtypes: int64(3), object(1)
memory usage: 40.1+ MB


In [73]:
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz")
weather['air_temperature_diff1'] = weather['air_temperature'].diff()
weather.at[0,'air_temperature_diff1'] = weather.at[1,'air_temperature_diff1']
weather['air_temperature_diff2'] = weather['air_temperature_diff1'].diff()
weather.at[0,'air_temperature_diff2'] = weather.at[1,'air_temperature_diff2']

In [74]:
buildings = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz',
                        usecols=["site_id", "building_id"])
results = pd.merge(left=results, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
results = results.set_index(['site_id','timestamp'])
weather = weather.set_index(['site_id','timestamp'])
results = pd.merge(left=results,right=weather,how='left',left_index=True,right_index=True)
results.reset_index(inplace=True)
results = results.drop(['site_id','meter'],axis=1)
results = reduce_mem_usage(results)

Потребление памяти меньше на 65.16 Мб (минус 67.7 %)


In [75]:
results['hour'] = results['timestamp'].dt.hour.astype('int8')
results['weekday'] = results['timestamp'].dt.weekday.astype('int8')
results['week'] = results['timestamp'].dt.week.astype('int8')
results['month'] = results['timestamp'].dt.month.astype('int8')
results['date'] = pd.to_datetime(results['timestamp'].dt.date)


dates_range = pd.date_range(start='2015-12-31',end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),end=dates_range.max())
results['is_holiday'] = results['date'].isin(us_holidays).astype('int8')
for weekday in range(0,7):
    results['is_wday' + str(weekday)] = results['weekday'].isin([weekday]).astype('int8')
for week in range(1,54):
    results['is_w' + str(week)] = results['week'].isin([week]).astype('int8')
for month in range(1,13):
    results['is_m' + str(month)] = results['month'].isin([month]).astype('int8')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [76]:
final = results.apply(calculate_model,axis=1,result_type='expand')

In [78]:
len(final)

1051200